In [1]:
import pandas as pd
import numpy as np
import re
import subprocess
from pathlib import Path 
import io
import dvc.api, dvc.repo

In [2]:
df = pd.read_csv("Google-Playstore.csv")
df.head()

,App Name,App Id,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,Price,...,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,Scraped Time
0,Gakondo,com.ishakwe.gakondo,Adventure,0.0,0.0,10+,10.0,15,True,0.0,...,https://beniyizibyose.tk/#/,jean21101999@gmail.com,"Feb 26, 2020","Feb 26, 2020",Everyone,https://beniyizibyose.tk/projects/,False,False,False,2021-06-15 20:19:35
1,Ampere Battery Info,com.webserveis.batteryinfo,Tools,4.4,64.0,"5,000+",5000.0,7662,True,0.0,...,https://webserveis.netlify.app/,webserveis@gmail.com,"May 21, 2020","May 06, 2021",Everyone,https://dev4phones.wordpress.com/licencia-de-uso/,True,False,False,2021-06-15 20:19:35
2,Vibook,com.doantiepvien.crm,Productivity,0.0,0.0,50+,50.0,58,True,0.0,...,NaN,vnacrewit@gmail.com,"Aug 9, 2019","Aug 19, 2019",Everyone,https://www.vietnamairlines.com/vn/en/terms-an...,False,False,False,2021-06-15 20:19:35
3,Smart City Trichy Public Service Vehicles 17UC...,cst.stJoseph.ug17ucs548,Communication,5.0,5.0,10+,10.0,19,True,0.0,...,http://www.climatesmarttech.com/,climatesmarttech2@gmail.com,"Sep 10, 2018","Oct 13, 2018",Everyone,NaN,True,False,False,2021-06-15 20:19:35
4,GROW.me,com.horodyski.grower,Tools,0.0,0.0,100+,100.0,478,True,0.0,...,http://www.horodyski.com.pl,rmilekhorodyski@gmail.com,"Feb 21, 2020","Nov 12, 2018",Everyone,http://www.horodyski.com.pl,False,False,False,2021-06-15 20:19:35


In [3]:
df.shape

(2312944, 24)

In [4]:
df.columns

Index(['App Name', 'App Id', 'Category', 'Rating', 'Rating Count', 'Installs',
       'Minimum Installs', 'Maximum Installs', 'Free', 'Price', 'Currency',
       'Size', 'Minimum Android', 'Developer Id', 'Developer Website',
       'Developer Email', 'Released', 'Last Updated', 'Content Rating',
       'Privacy Policy', 'Ad Supported', 'In App Purchases', 'Editors Choice',
       'Scraped Time'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2312944 entries, 0 to 2312943
Data columns (total 24 columns):
 #   Column             Dtype  
---  ------             -----  
 0   App Name           object 
 1   App Id             object 
 2   Category           object 
 3   Rating             float64
 4   Rating Count       float64
 5   Installs           object 
 6   Minimum Installs   float64
 7   Maximum Installs   int64  
 8   Free               bool   
 9   Price              float64
 10  Currency           object 
 11  Size               object 
 12  Minimum Android    object 
 13  Developer Id       object 
 14  Developer Website  object 
 15  Developer Email    object 
 16  Released           object 
 17  Last Updated       object 
 18  Content Rating     object 
 19  Privacy Policy     object 
 20  Ad Supported       bool   
 21  In App Purchases   bool   
 22  Editors Choice     bool   
 23  Scraped Time       object 
dtypes: bool(4), float64(4), int64(1), object(15)
memor

In [6]:
len(df['App Id'].unique())

2312944

In [7]:
df.describe()

,Rating,Rating Count,Minimum Installs,Maximum Installs,Price
count,2.290061e+06,2.290061e+06,2.312837e+06,2.312944e+06,2.312944e+06
mean,2.203152e+00,2.864839e+03,1.834452e+05,3.202017e+05,1.034992e-01
std,2.106223e+00,2.121626e+05,1.513144e+07,2.355495e+07,2.633127e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,5.000000e+01,8.400000e+01,0.000000e+00
50%,2.900000e+00,6.000000e+00,5.000000e+02,6.950000e+02,0.000000e+00
75%,4.300000e+00,4.200000e+01,5.000000e+03,7.354000e+03,0.000000e+00
max,5.000000e+00,1.385576e+08,1.000000e+10,1.205763e+10,4.000000e+02


### Currency

In [8]:
df['Currency'].value_counts()

Currency
USD    2311548
XXX       1236
EUR          6
INR          5
GBP          3
CAD          2
VND          1
BRL          1
KRW          1
TRY          1
RUB          1
SGD          1
AUD          1
PKR          1
ZAR          1
Name: count, dtype: int64

In [9]:
df.drop(columns=['Installs', 'Minimum Installs', 'Currency', 'Developer Id', 'Developer Website', 'Developer Email',
                 'Privacy Policy', 'Scraped Time'], axis=1, inplace=True)

In [10]:
df.head()

,App Name,App Id,Category,Rating,Rating Count,Maximum Installs,Free,Price,Size,Minimum Android,Released,Last Updated,Content Rating,Ad Supported,In App Purchases,Editors Choice
0,Gakondo,com.ishakwe.gakondo,Adventure,0.0,0.0,15,True,0.0,10M,7.1 and up,"Feb 26, 2020","Feb 26, 2020",Everyone,False,False,False
1,Ampere Battery Info,com.webserveis.batteryinfo,Tools,4.4,64.0,7662,True,0.0,2.9M,5.0 and up,"May 21, 2020","May 06, 2021",Everyone,True,False,False
2,Vibook,com.doantiepvien.crm,Productivity,0.0,0.0,58,True,0.0,3.7M,4.0.3 and up,"Aug 9, 2019","Aug 19, 2019",Everyone,False,False,False
3,Smart City Trichy Public Service Vehicles 17UC...,cst.stJoseph.ug17ucs548,Communication,5.0,5.0,19,True,0.0,1.8M,4.0.3 and up,"Sep 10, 2018","Oct 13, 2018",Everyone,True,False,False
4,GROW.me,com.horodyski.grower,Tools,0.0,0.0,478,True,0.0,6.2M,4.1 and up,"Feb 21, 2020","Nov 12, 2018",Everyone,False,False,False


## Minimum android

In [11]:
df['Minimum Android'].value_counts()

Minimum Android
4.1 and up      604465
5.0 and up      396998
4.4 and up      390311
4.0.3 and up    180482
4.0 and up      153441
                 ...  
2.3 - 3.2            1
6.0 - 7.1.1          1
2.3 - 8.0            1
3.0 - 7.1.1          1
2.3.3 - 6.0          1
Name: count, Length: 154, dtype: int64

In [12]:
df.drop(columns='Minimum Android', axis=1, inplace=True)

### Size

In [13]:
df['Size'].value_counts()

Size
Varies with device    74777
11M                   62157
12M                   56080
13M                   48034
14M                   45211
                      ...  
8.7k                      1
784M                      1
385M                      1
7.6k                      1
512M                      1
Name: count, Length: 1657, dtype: int64

In [14]:
def convert_to_megabytes(size):
    """Converts a size string from KB to MB"""
    # Ensure the size is a string
    size_str = str(size)

    if size_str.endswith('k'):
        # Remove the 'k' and commas, then convert to float
        size_in_kb = float(size_str[:-1].replace(',', ''))
        # Convert to MB by dividing by 1024
        size_in_mb = size_in_kb / 1024
        return f"{size_in_mb:.2f}M"  # Format to 2 decimal places and add 'M'
    else:
        return size_str.replace(',', '')

df['Size'] = df['Size'].apply(convert_to_megabytes)

In [15]:
df['Size'].value_counts()

Size
Varies with device    74777
11M                   62157
12M                   56080
13M                   48034
14M                   45211
                      ...  
347M                      1
606M                      1
412M                      1
940M                      1
512M                      1
Name: count, Length: 709, dtype: int64

In [16]:
def convert_size(size):
    if 'M' in size:
        return float(size.replace('M', ''))
    elif 'k' in size:
        return float(size.replace('k', '')) / 1024
    return np.nan
df['Size_num'] = df['Size'].apply(lambda x: convert_size(x) if x != 'Varies with device' else np.nan)
# Calculate average size per category
df_sorted = df.sort_values(by='Size_num', ascending=False)
top_5_per_category = df_sorted.groupby('Category').head(3)
category_avg_size = top_5_per_category.groupby('Category')['Size_num'].mean().round(1)
print(category_avg_size)
# Replace 'Varies with device' with the average size of the category
df['Size_num'] = df.apply(
    lambda row: category_avg_size[row['Category']] if pd.isna(row['Size_num']) else row['Size_num'],
    axis=1
)

Category
Action                     866.0
Adventure                  804.7
Arcade                     431.0
Art & Design               477.7
Auto & Vehicles            542.0
Beauty                     229.3
Board                      269.3
Books & Reference          987.3
Business                   443.7
Card                       461.7
Casino                     206.7
Casual                     300.3
Comics                     166.7
Communication              530.7
Dating                     229.3
Education                  630.0
Educational                541.7
Entertainment              795.3
Events                     362.7
Finance                    362.7
Food & Drink               276.3
Health & Fitness           589.3
House & Home               387.0
Libraries & Demo           199.0
Lifestyle                  463.7
Maps & Navigation          343.7
Medical                    663.7
Music                      550.7
Music & Audio              323.7
News & Magazines           184.7
P

In [17]:
# Convert the numeric size back to string with 'M' for display purposes
df['Size'] = df['Size_num'].apply(lambda x: f"{x:.1f}M")
# Drop the helper column
df.drop(columns=['Size_num'], inplace=True)

In [18]:
def convert_size_to_float(size):
    if 'M' in size:
        return float(size.replace('M', ''))
df['Size'] = df['Size'].apply(convert_size_to_float)

In [19]:
df.loc[df['Size'] == 'Varies with device'].shape[0]

0

In [20]:
df.rename(columns={'Size':'Size_M'}, inplace=True)
df.head()

,App Name,App Id,Category,Rating,Rating Count,Maximum Installs,Free,Price,Size_M,Released,Last Updated,Content Rating,Ad Supported,In App Purchases,Editors Choice
0,Gakondo,com.ishakwe.gakondo,Adventure,0.0,0.0,15,True,0.0,10.0,"Feb 26, 2020","Feb 26, 2020",Everyone,False,False,False
1,Ampere Battery Info,com.webserveis.batteryinfo,Tools,4.4,64.0,7662,True,0.0,2.9,"May 21, 2020","May 06, 2021",Everyone,True,False,False
2,Vibook,com.doantiepvien.crm,Productivity,0.0,0.0,58,True,0.0,3.7,"Aug 9, 2019","Aug 19, 2019",Everyone,False,False,False
3,Smart City Trichy Public Service Vehicles 17UC...,cst.stJoseph.ug17ucs548,Communication,5.0,5.0,19,True,0.0,1.8,"Sep 10, 2018","Oct 13, 2018",Everyone,True,False,False
4,GROW.me,com.horodyski.grower,Tools,0.0,0.0,478,True,0.0,6.2,"Feb 21, 2020","Nov 12, 2018",Everyone,False,False,False


In [21]:
df.rename(columns={'Maximum Installs':'Installs'}, inplace=True)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2312944 entries, 0 to 2312943
Data columns (total 15 columns):
 #   Column            Dtype  
---  ------            -----  
 0   App Name          object 
 1   App Id            object 
 2   Category          object 
 3   Rating            float64
 4   Rating Count      float64
 5   Installs          int64  
 6   Free              bool   
 7   Price             float64
 8   Size_M            float64
 9   Released          object 
 10  Last Updated      object 
 11  Content Rating    object 
 12  Ad Supported      bool   
 13  In App Purchases  bool   
 14  Editors Choice    bool   
dtypes: bool(4), float64(4), int64(1), object(6)
memory usage: 202.9+ MB


### Dropping Nulls

In [23]:
df['Released'].isna().sum()

71053

In [24]:
df['App Name'].isna().sum()

5

In [25]:
df['App Id'].isna().sum()

0

In [26]:
df['Category'].isna().sum()

0

In [27]:
df['Rating'].isna().sum()

22883

In [28]:
df['Rating Count'].isna().sum()

22883

In [29]:
df['Installs'].isna().sum()

0

In [30]:
df['Free'].isna().sum()

0

In [31]:
df['Price'].isna().sum()

0

In [32]:
df['Size_M'].isna().sum()

0

In [33]:
df['Content Rating'].isna().sum()

0

In [34]:
df['Ad Supported'].isna().sum()

0

In [35]:
df['In App Purchases'].isna().sum()

0

In [36]:
df['Editors Choice'].isna().sum()

0

#### Nulls in [App Name: 5 | Rating and Rating count: 22883 | Released: 71053]

In [37]:
df = df.dropna(subset=['App Name'], how='all')

In [38]:
df['App Name'].isna().sum()

0

In [39]:
df = df.dropna(subset=['Rating'], how='all')

In [40]:
df['Rating'].isna().sum()

0

In [41]:
df['Rating Count'].isna().sum()

0

In [42]:
df = df.dropna(subset=['Released'], how='all')

In [43]:
df['Released'].isna().sum()

0

In [44]:
df.head()

,App Name,App Id,Category,Rating,Rating Count,Installs,Free,Price,Size_M,Released,Last Updated,Content Rating,Ad Supported,In App Purchases,Editors Choice
0,Gakondo,com.ishakwe.gakondo,Adventure,0.0,0.0,15,True,0.0,10.0,"Feb 26, 2020","Feb 26, 2020",Everyone,False,False,False
1,Ampere Battery Info,com.webserveis.batteryinfo,Tools,4.4,64.0,7662,True,0.0,2.9,"May 21, 2020","May 06, 2021",Everyone,True,False,False
2,Vibook,com.doantiepvien.crm,Productivity,0.0,0.0,58,True,0.0,3.7,"Aug 9, 2019","Aug 19, 2019",Everyone,False,False,False
3,Smart City Trichy Public Service Vehicles 17UC...,cst.stJoseph.ug17ucs548,Communication,5.0,5.0,19,True,0.0,1.8,"Sep 10, 2018","Oct 13, 2018",Everyone,True,False,False
4,GROW.me,com.horodyski.grower,Tools,0.0,0.0,478,True,0.0,6.2,"Feb 21, 2020","Nov 12, 2018",Everyone,False,False,False


### Released

In [45]:
df['Released'] = pd.to_datetime(df['Released'])

In [46]:
df['Released_Year'] = df['Released'].dt.year
df['Released_Month'] = df['Released'].dt.month

In [47]:
df.head()

,App Name,App Id,Category,Rating,Rating Count,Installs,Free,Price,Size_M,Released,Last Updated,Content Rating,Ad Supported,In App Purchases,Editors Choice,Released_Year,Released_Month
0,Gakondo,com.ishakwe.gakondo,Adventure,0.0,0.0,15,True,0.0,10.0,2020-02-26,"Feb 26, 2020",Everyone,False,False,False,2020,2
1,Ampere Battery Info,com.webserveis.batteryinfo,Tools,4.4,64.0,7662,True,0.0,2.9,2020-05-21,"May 06, 2021",Everyone,True,False,False,2020,5
2,Vibook,com.doantiepvien.crm,Productivity,0.0,0.0,58,True,0.0,3.7,2019-08-09,"Aug 19, 2019",Everyone,False,False,False,2019,8
3,Smart City Trichy Public Service Vehicles 17UC...,cst.stJoseph.ug17ucs548,Communication,5.0,5.0,19,True,0.0,1.8,2018-09-10,"Oct 13, 2018",Everyone,True,False,False,2018,9
4,GROW.me,com.horodyski.grower,Tools,0.0,0.0,478,True,0.0,6.2,2020-02-21,"Nov 12, 2018",Everyone,False,False,False,2020,2


In [48]:
df['Released'].max()

Timestamp('2021-06-16 00:00:00')

In [49]:
# Ensure both columns are in datetime format
# specific_date = pd.to_datetime('2021-06-16')
df['Released'] = pd.to_datetime(df['Released'])
df['Last Updated'] = pd.to_datetime(df['Last Updated'])

# Calculate the difference in days
df['Days_Between'] = np.abs((df['Last Updated'] - df['Released']).dt.days)

one_month_days = 30

df = df[(df['Days_Between'] >= 0) & (df['Days_Between'] <= one_month_days)]

# Calculate the daily average number of installs
# Use np.where to handle cases where Days_Between is zero
df['Daily_Avg_Installs'] = np.where(
    df['Days_Between'] == 0, 
    df['Installs'], 
    (df['Installs'] / df['Days_Between']).round(1)
)

In [50]:
df.head()

,App Name,App Id,Category,Rating,Rating Count,Installs,Free,Price,Size_M,Released,Last Updated,Content Rating,Ad Supported,In App Purchases,Editors Choice,Released_Year,Released_Month,Days_Between,Daily_Avg_Installs
0,Gakondo,com.ishakwe.gakondo,Adventure,0.0,0.0,15,True,0.0,10.0,2020-02-26,2020-02-26,Everyone,False,False,False,2020,2,0,15.0
2,Vibook,com.doantiepvien.crm,Productivity,0.0,0.0,58,True,0.0,3.7,2019-08-09,2019-08-19,Everyone,False,False,False,2019,8,10,5.8
6,unlimited 4G data prank free app,getfreedata.superfatiza.unlimitedjiodataprank,Libraries & Demo,4.5,12.0,2567,True,0.0,2.5,2019-09-23,2019-09-27,Everyone,True,False,False,2019,9,4,641.8
7,The Everyday Calendar,com.mozaix.simoneboard,Lifestyle,2.0,39.0,702,True,0.0,16.0,2019-06-21,2019-06-21,Everyone,False,False,False,2019,6,0,702.0
10,Dodge The Cars!,com.MrScratchEnterprises.CarDogeGame,Racing,4.9,55.0,329,True,0.0,51.0,2020-07-30,2020-07-30,Everyone,False,False,False,2020,7,0,329.0


In [56]:
df.iloc[:600000]

,App Name,App Id,Category,Rating,Rating Count,Installs,Free,Price,Size_M,Released,Last Updated,Content Rating,Ad Supported,In App Purchases,Editors Choice,Released_Year,Released_Month,Days_Between,Daily_Avg_Installs
0,Gakondo,com.ishakwe.gakondo,Adventure,0.0,0.0,15,True,0.0,10.0,2020-02-26,2020-02-26,Everyone,False,False,False,2020,2,0,15.0
2,Vibook,com.doantiepvien.crm,Productivity,0.0,0.0,58,True,0.0,3.7,2019-08-09,2019-08-19,Everyone,False,False,False,2019,8,10,5.8
6,unlimited 4G data prank free app,getfreedata.superfatiza.unlimitedjiodataprank,Libraries & Demo,4.5,12.0,2567,True,0.0,2.5,2019-09-23,2019-09-27,Everyone,True,False,False,2019,9,4,641.8
7,The Everyday Calendar,com.mozaix.simoneboard,Lifestyle,2.0,39.0,702,True,0.0,16.0,2019-06-21,2019-06-21,Everyone,False,False,False,2019,6,0,702.0
10,Dodge The Cars!,com.MrScratchEnterprises.CarDogeGame,Racing,4.9,55.0,329,True,0.0,51.0,2020-07-30,2020-07-30,Everyone,False,False,False,2020,7,0,329.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1616616,GeorgeApp LogBook,eu.georgeapp.georgeapplogbook,Sports,0.0,0.0,214,True,0.0,2.2,2018-04-29,2018-04-29,Everyone,False,False,False,2018,4,0,214.0
1616618,Drawer Super Racer - DRAWING GAME,com.BoomaStudio.DrawerSuperRacer,Arcade,0.0,0.0,15,True,0.0,20.0,2021-03-28,2021-04-06,Everyone,True,False,False,2021,3,9,1.7
1616622,Bahasa Indonesia Kelas 12 Kurikulum 2013,com.reverdroid.bahasaindonesiakelas12,Books & Reference,4.6,50.0,17423,True,0.0,77.0,2018-08-27,2018-08-28,Everyone,True,False,False,2018,8,1,17423.0
1616631,Video Voice Changer For Short Video Makers,com.stupendous.videovoice.changer.dp,Video Players & Editors,2.8,120.0,32306,True,0.0,26.0,2020-03-28,2020-03-28,Everyone,True,True,False,2020,3,0,32306.0


In [58]:
df = df.iloc[:600000]

In [59]:
filepath = Path('Google-Playstore.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(filepath, index=False)